In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


You may need to restart session after installing bitsandbytes

In [ ]:
!pip install -U bitsandbytes accelerate transformers

In [ ]:
!export PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True

In [ ]:
!pip install faiss-cpu
!pip install fuzzywuzzy

In [ ]:
!pip install sentence-transformers

In [ ]:
import pickle
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import faiss
from transformers import DPRQuestionEncoderTokenizer,DPRQuestionEncoder,T5Tokenizer, T5ForConditionalGeneration
import torch
from fuzzywuzzy import process

In [ ]:
import os
from transformers import AutoModelForCausalLM, AutoTokenizer,AutoConfig,BitsAndBytesConfig
from accelerate import init_empty_weights, infer_auto_device_map
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

In [ ]:
question_tokenizer = DPRQuestionEncoderTokenizer.from_pretrained("facebook/dpr-question_encoder-single-nq-base")
question_encoder = DPRQuestionEncoder.from_pretrained("facebook/dpr-question_encoder-single-nq-base")

Some weights of the model checkpoint at facebook/dpr-question_encoder-single-nq-base were not used when initializing DPRQuestionEncoder: ['question_encoder.bert_model.pooler.dense.bias', 'question_encoder.bert_model.pooler.dense.weight']
- This IS expected if you are initializing DPRQuestionEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DPRQuestionEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
path = '/content/drive/MyDrive/Data mining assignment/Pickled/'

with open(path+'context_embeddings_findings.pkl','rb') as f:
  context_embedding = pickle.load(f)

with open(path+'tfidf_matrix_findings.pkl','rb') as f:
  tfidf_matrix = pickle.load(f)

with open(path+'tfidf_vectorizer_findings.pkl','rb') as f:
  tfidf_vectorizer = pickle.load(f)

/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator TfidfTransformer from version 1.2.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/base.py:380: InconsistentVersionWarning: Trying to unpickle estimator TfidfVectorizer from version 1.2.2 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [ ]:
document_df = pd.read_csv(path+'dataset.csv')

In [ ]:
document_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2014 entries, 0 to 2013
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   pubmed_id  2014 non-null   object
 1   findings   2014 non-null   object
dtypes: object(2)
memory usage: 31.6+ KB


In [ ]:
def sparse_retrieval(query, documentDf = document_df,top_k=5):
    query_vec = tfidf_vectorizer.transform([query])

    scores = cosine_similarity(query_vec, tfidf_matrix)[0]

    top_indices = scores.argsort()[-top_k:][::-1]
    print(top_indices)
    return documentDf.iloc[top_indices]

In [ ]:
faiss_index = faiss.IndexFlatIP(context_embedding.shape[1])
faiss_index.add(context_embedding)

def dense_retrieval(query,documentDf = document_df ,top_k=5):
    query_inputs = question_tokenizer(query, return_tensors="pt", truncation=True, max_length=512)
    query_embedding = question_encoder(**query_inputs).pooler_output.detach().numpy()
    scores, indices = faiss_index.search(query_embedding, top_k)
    print(indices[0])

    return documentDf.iloc[indices[0]]

In [ ]:
userQuery = 'what is covid?'

In [ ]:
sr_documents = sparse_retrieval(userQuery)
dr_documents = dense_retrieval(userQuery)

[ 753  565 1231 1573   33]
[1162 1611  141 1701 1231]


In [ ]:
device = torch.device('cpu')

In [ ]:
from sentence_transformers import SentenceTransformer, util
sentence_t5_model = SentenceTransformer('sentence-transformers/sentence-t5-large')

In [ ]:
def re_rank_documents_sentence_t5(query, documents):
    """
    Ranks a list of documents based on their relevance to a query using Sentence-T5.
    Code was assisted by ChatGPT
    """
    # Encode the query and documents
    query_embedding = sentence_t5_model.encode(query, convert_to_tensor=True)
    document_embeddings = sentence_t5_model.encode([doc for doc in documents], convert_to_tensor=True)

    # Compute cosine similarity between the query and each document
    similarity_scores = util.cos_sim(query_embedding, document_embeddings)

    # Pair documents with their similarity scores
    document_scores = [(doc, score.item()) for doc, score in zip(documents, similarity_scores[0])]

    # Sort documents by their similarity scores in descending order
    ranked_documents = sorted(document_scores, key=lambda x: x[1], reverse=True)

    return ranked_documents

In [ ]:
def retrieve_and_rank(query, xr_documents):
  xr_ranked_documents_st_t5 = re_rank_documents_sentence_t5(userQuery, xr_documents['findings'].to_numpy())
  re_rank_xr_st_t5 = pd.DataFrame(columns=document_df.columns)

  for idx, (doc, score) in enumerate(xr_ranked_documents_st_t5, 1):
      best_match = process.extractOne(doc, xr_documents['findings'])
      print(doc)
      matched_row = xr_documents[xr_documents['findings'] == best_match[0]]
      re_rank_xr_st_t5 = pd.concat([re_rank_xr_st_t5, matched_row], ignore_index=True)
      print("Best match:", best_match)
      print("\n")
      print(f"{idx}. {doc}")
      print("\n")
      print(f"Score:{score:.4f}\n")
  return re_rank_xr_st_t5

In [ ]:
re_rank_sr_st_t5 = retrieve_and_rank(userQuery, sr_documents)
re_rank_dr_st_t5 = retrieve_and_rank(userQuery, dr_documents)

as of february 20, 2020, there have been over 72,000 confirmed cases of covid-19, with the majority in china. discrimination and prejudice, driven by fear or misinformation, have emerged in response to the outbreak. this has resulted in innocent people in china being unfairly blamed, and similar incidents have occurred in other countries. the formal names for the virus and disease have been established by the international committee on taxonomy of viruses and the world health organization, respectively. the outbreak has highlighted the negative side of human societies, as prejudice and bigotry have occurred with each huge epidemic in history. fear, prejudice, and discrimination can impede anti-covid-19 efforts, as they can discourage people from seeking medical help and prevent effective control measures from being implemented. open and transparent information, as well as administrative warnings, are necessary to reduce fear and discrimination.
Best match: ('as of february 20, 2020, th

In [ ]:
re_rank_sr_st_t5

,pubmed_id,findings
0,pmc7052559,"as of february 20, 2020, there have been over ..."
1,pmc7067954,"a new strain of the coronavirus, sars-cov-2, i..."
2,pmc4171117,the liu-hui-wen model predicts that 15-30% of ...
3,pmc2725414,a study published in the journal of the americ...
4,pmc6558557,in the context of disease surveillance systems...


In [ ]:
re_rank_dr_st_t5

,pubmed_id,findings
0,pmc7052559,"as of february 20, 2020, there have been over ..."
1,pmc5572892,in a study analyzing the feces of laboratory f...
2,pmc6810513,covs human coronaviruses are a major cause o...
3,pmc1492161,intended audience 5-minute clinical consults ...
4,pmc6716890,"between 2009 and 2018, there were 1713 article..."


In [ ]:
re_rank_sr_st_t5

,pubmed_id,findings
0,pmc7052559,"as of february 20, 2020, there have been over ..."
1,pmc7067954,"a new strain of the coronavirus, sars-cov-2, i..."
2,pmc4171117,the liu-hui-wen model predicts that 15-30% of ...
3,pmc2725414,a study published in the journal of the americ...
4,pmc6558557,in the context of disease surveillance systems...


In [ ]:
re_rank_dr_st_t5

,pubmed_id,findings
0,pmc7052559,"as of february 20, 2020, there have been over ..."
1,pmc5572892,in a study analyzing the feces of laboratory f...
2,pmc6810513,covs human coronaviruses are a major cause o...
3,pmc1492161,intended audience 5-minute clinical consults ...
4,pmc6716890,"between 2009 and 2018, there were 1713 article..."


## RAG Flow Development

In [ ]:
import gc
torch.cuda.empty_cache()
gc.collect()
torch.cuda.ipc_collect()  # Collect GPU inter-process communication memory

In [ ]:
model_name = "HuggingFaceH4/zephyr-7b-beta"
tokenizer_zephyr = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# model was too large ~16GB > 15GB max
# so ChatGPT suggested to use 4-bit quantization to further reduce memory usage
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_use_double_quant=True)

model_zephyr = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    quantization_config = quant_config,
    device_map='cuda'
)

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model_zephyr.to(device)

MistralForCausalLM(
  (model): MistralModel(
    (embed_tokens): Embedding(32000, 4096, padding_idx=2)
    (layers): ModuleList(
      (0-31): 32 x MistralDecoderLayer(
        (self_attn): MistralAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
        )
        (mlp): MistralMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): MistralRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): MistralRMSNorm((4096,), eps=1e-05)
      )
    )
 

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [ ]:
from collections import Counter
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
def extract_key_phrases(text):
  """
  Extracts key phrases from the given text using a simple frequency-based approach.
    Code was assisted by ChatGPT

  Args:
    text: The input text string.

  Returns:
    list: A list of key phrases.
  """

  # Preprocess the text
  stop_words = set(stopwords.words('english'))
  words = word_tokenize(text.lower())
  words = [word for word in words if word.isalnum() and word not in stop_words]

  # Create n-grams (consider bigrams and trigrams)
  phrases = []
  for i in range(len(words) - 1):
    phrases.append(" ".join(words[i:i+2]))  # Bigrams
    if i < len(words) - 2:
      phrases.append(" ".join(words[i:i+3]))  # Trigrams

  # Count phrase frequencies
  phrase_counts = Counter(phrases)

  # Select top-k frequent phrases (adjust k as needed)
  top_phrases = [phrase for phrase, count in phrase_counts.most_common(5)]

  return top_phrases

def augment_and_generate(re_rank_xr_st_t5, query, enable_query_augmentation = True):
  """Augments the query with relevant information from retrieved documents and generates a response.

  Args:
    re_rank_xr_st_t5: DataFrame containing retrieved and ranked documents.
    query: Original user query.

  Returns:
    str: Generated response.
  """

  augmented_query = query

  if enable_query_augmentation:
    # Extract key phrases from top-ranked documents
    top_documents = re_rank_xr_st_t5.head(3)  # Consider top 3 documents
    key_phrases = []
    for index, row in top_documents.iterrows():
      key_phrases.extend(extract_key_phrases(row['findings']))

    augmented_query = f"{query} related to: {', '.join(key_phrases)}"

  contexts = "\n  ".join([f"[{index}] {row['findings']}" for index, row in re_rank_xr_st_t5.iterrows()])
  prompt_template = f"""<|system|>
  # This is a chat between a user and an AI assistant. The assistant follows strict rules when answering based on up to 5 provided contexts:

  # 1. If the answer is found in the contexts, the assistant must provide a detailed response **with citations at the end of each point**.
  # 2. The assistant must not add information beyond the provided contexts.
  # 3. If no relevant answer is found, the assistant must respond with **'Cannot extract findings from the provided contexts.'** and **'Summary is not available'**.
  # 4. **Citations must be formatted in IEEE style within square brackets []** at the end of each point.
  # 5. Answers must be structured as a numbered list and **each point must contain its respective citation.**
  # 6. Responses must be ranked according to the provided contexts.

  # **Example format when findings can be extracted from the provided contexts:**
  # Findings:
  # **Example format when findings are available:**
  # 1. Lorem ipsum dolor sit amet, consectetur adipiscing elit. Nulla laoreet odio a arcu ultricies rutrum. [1].
  # 2. Nam laoreet lorem consequat ipsum cursus consequat. Donec augue risus, cursus nec accumsan a, rutrum vel urna. [1].
  # ...
  # 20. Aliquam quis felis consectetur, imperdiet mi vehicula, malesuada velit. [5].

  # **Example format when findings cannot be extracted from the provided contexts:**
  # Findings:
  #   Cannot extract findings from the provided contexts.
  # </s>

  # <|user|>
  # Contexts:
  #   {contexts}

  # Question: Summarize 20 key findings **strictly from the provided contexts** that are **relevant to answering the query** `{augmented_query}`.
  # - **Only use information explicitly found in the provided contexts.**
  # - **If no relevant information is found, respond with 'No answer found.'**
  # - **Each point must include an IEEE-style citation in square brackets [] from the provided contexts.**
  # - **Do not infer, speculate, or generate information beyond what is explicitly stated.**
  # - **Structure the response as a numbered list.**
  # </s>
  # <|assistant|>
  # """
  # Tokenize and generate response
  inputs = tokenizer_zephyr(prompt_template, return_tensors="pt").to("cuda")
  inputs = {key: value.to(device) for key, value in inputs.items()}

  # Ensure no gradients are calculated (saves memory)
  with torch.inference_mode(), torch.amp.autocast('cuda'):
    outputs = model_zephyr.generate(
          **inputs,
          max_new_tokens=2000,
          do_sample=True,
          top_k=5,
          top_p=0.95,
          temperature=0.7,  # Control randomness of predictions
          num_return_sequences=1,
          pad_token_id=tokenizer_zephyr.eos_token_id # Return a single result
    )

  # return response from zephyr
  response = tokenizer_zephyr.decode(outputs[0], skip_special_tokens=True)
  print(response)
  return response


In [ ]:
response_dr_augment_T = augment_and_generate(re_rank_dr_st_t5, userQuery, True)

<|system|>
  # This is a chat between a user and an AI assistant. The assistant follows strict rules when answering based on up to 5 provided contexts:

  # 1. If the answer is found in the contexts, the assistant must provide a detailed response **with citations at the end of each point**.
  # 2. The assistant must not add information beyond the provided contexts.
  # 3. If no relevant answer is found, the assistant must respond with **'Cannot extract findings from the provided contexts.'** and **'Summary is not available'**.
  # 4. **Citations must be formatted in IEEE style within square brackets []** at the end of each point.
  # 5. Answers must be structured as a numbered list and **each point must contain its respective citation.**
  # 6. Responses must be ranked according to the provided contexts.

  # **Example format when findings can be extracted from the provided contexts:**
  # Findings:
  # **Example format when findings are available:**
  # 1. Lorem ipsum dolor sit amet, 

In [ ]:
response_dr_augment_F = augment_and_generate(re_rank_dr_st_t5, userQuery, False)

<|system|>
  # This is a chat between a user and an AI assistant. The assistant follows strict rules when answering based on up to 5 provided contexts:

  # 1. If the answer is found in the contexts, the assistant must provide a detailed response **with citations at the end of each point**.
  # 2. The assistant must not add information beyond the provided contexts.
  # 3. If no relevant answer is found, the assistant must respond with **'Cannot extract findings from the provided contexts.'** and **'Summary is not available'**.
  # 4. **Citations must be formatted in IEEE style within square brackets []** at the end of each point.
  # 5. Answers must be structured as a numbered list and **each point must contain its respective citation.**
  # 6. Responses must be ranked according to the provided contexts.

  # **Example format when findings can be extracted from the provided contexts:**
  # Findings:
  # **Example format when findings are available:**
  # 1. Lorem ipsum dolor sit amet, 

In [ ]:
response_sr_augment_T = augment_and_generate(re_rank_sr_st_t5, userQuery, True)

<|system|>
  # This is a chat between a user and an AI assistant. The assistant follows strict rules when answering based on up to 5 provided contexts:

  # 1. If the answer is found in the contexts, the assistant must provide a detailed response **with citations at the end of each point**.
  # 2. The assistant must not add information beyond the provided contexts.
  # 3. If no relevant answer is found, the assistant must respond with **'Cannot extract findings from the provided contexts.'** and **'Summary is not available'**.
  # 4. **Citations must be formatted in IEEE style within square brackets []** at the end of each point.
  # 5. Answers must be structured as a numbered list and **each point must contain its respective citation.**
  # 6. Responses must be ranked according to the provided contexts.

  # **Example format when findings can be extracted from the provided contexts:**
  # Findings:
  # **Example format when findings are available:**
  # 1. Lorem ipsum dolor sit amet, 

In [ ]:
response_sr_augment_F = augment_and_generate(re_rank_sr_st_t5, userQuery, False)

<|system|>
  # This is a chat between a user and an AI assistant. The assistant follows strict rules when answering based on up to 5 provided contexts:

  # 1. If the answer is found in the contexts, the assistant must provide a detailed response **with citations at the end of each point**.
  # 2. The assistant must not add information beyond the provided contexts.
  # 3. If no relevant answer is found, the assistant must respond with **'Cannot extract findings from the provided contexts.'** and **'Summary is not available'**.
  # 4. **Citations must be formatted in IEEE style within square brackets []** at the end of each point.
  # 5. Answers must be structured as a numbered list and **each point must contain its respective citation.**
  # 6. Responses must be ranked according to the provided contexts.

  # **Example format when findings can be extracted from the provided contexts:**
  # Findings:
  # **Example format when findings are available:**
  # 1. Lorem ipsum dolor sit amet, 

## RAGs Full Flow

In [ ]:
def adrian_steve_rags(retrieval_type, query, should_augment_query):
  # retrieve
  if retrieval_type == 'sparse':
    xr_documents = sparse_retrieval(query)
  else:
    xr_documents = dense_retrieval(query)

  re_rank_xr_st_t5 = retrieve_and_rank(query, xr_documents)

  # augment & generate
  return augment_and_generate(re_rank_xr_st_t5, query, should_augment_query)


In [ ]:
responseSrCovid_1 = adrian_steve_rags('sparse', 'what are the common symptoms of covid-19?', True)

[ 753  565 1869 1231 1573]
as of february 20, 2020, there have been over 72,000 confirmed cases of covid-19, with the majority in china. discrimination and prejudice, driven by fear or misinformation, have emerged in response to the outbreak. this has resulted in innocent people in china being unfairly blamed, and similar incidents have occurred in other countries. the formal names for the virus and disease have been established by the international committee on taxonomy of viruses and the world health organization, respectively. the outbreak has highlighted the negative side of human societies, as prejudice and bigotry have occurred with each huge epidemic in history. fear, prejudice, and discrimination can impede anti-covid-19 efforts, as they can discourage people from seeking medical help and prevent effective control measures from being implemented. open and transparent information, as well as administrative warnings, are necessary to reduce fear and discrimination.
Best match: ('

In [ ]:
responseSrCovid_2 = adrian_steve_rags('sparse', 'what are the common symptoms of covid-19?', False)

[ 753  565 1869 1231 1573]
as of february 20, 2020, there have been over 72,000 confirmed cases of covid-19, with the majority in china. discrimination and prejudice, driven by fear or misinformation, have emerged in response to the outbreak. this has resulted in innocent people in china being unfairly blamed, and similar incidents have occurred in other countries. the formal names for the virus and disease have been established by the international committee on taxonomy of viruses and the world health organization, respectively. the outbreak has highlighted the negative side of human societies, as prejudice and bigotry have occurred with each huge epidemic in history. fear, prejudice, and discrimination can impede anti-covid-19 efforts, as they can discourage people from seeking medical help and prevent effective control measures from being implemented. open and transparent information, as well as administrative warnings, are necessary to reduce fear and discrimination.
Best match: ('

In [ ]:
responseDrCovid_1 = adrian_steve_rags('dense', 'what are the common symptoms of covid-19?', True)

[1231 1610  976  849 1162]
as of february 20, 2020, there have been over 72,000 confirmed cases of covid-19, with the majority in china. discrimination and prejudice, driven by fear or misinformation, have emerged in response to the outbreak. this has resulted in innocent people in china being unfairly blamed, and similar incidents have occurred in other countries. the formal names for the virus and disease have been established by the international committee on taxonomy of viruses and the world health organization, respectively. the outbreak has highlighted the negative side of human societies, as prejudice and bigotry have occurred with each huge epidemic in history. fear, prejudice, and discrimination can impede anti-covid-19 efforts, as they can discourage people from seeking medical help and prevent effective control measures from being implemented. open and transparent information, as well as administrative warnings, are necessary to reduce fear and discrimination.
Best match: ('

In [ ]:
responseDrCovid_2 = adrian_steve_rags('dense', 'what are the common symptoms of covid-19?', False)

[1231 1610  976  849 1162]
as of february 20, 2020, there have been over 72,000 confirmed cases of covid-19, with the majority in china. discrimination and prejudice, driven by fear or misinformation, have emerged in response to the outbreak. this has resulted in innocent people in china being unfairly blamed, and similar incidents have occurred in other countries. the formal names for the virus and disease have been established by the international committee on taxonomy of viruses and the world health organization, respectively. the outbreak has highlighted the negative side of human societies, as prejudice and bigotry have occurred with each huge epidemic in history. fear, prejudice, and discrimination can impede anti-covid-19 efforts, as they can discourage people from seeking medical help and prevent effective control measures from being implemented. open and transparent information, as well as administrative warnings, are necessary to reduce fear and discrimination.
Best match: ('

## Query borderline-related [Dataset is focused on Health, and a political figure such as Donald Trump may be related in some indirect way]

In [ ]:
responseSrTrump_1 = adrian_steve_rags('sparse', 'who is Donald Trump?', True)

[2013  675  662  663  664]
in a study by capelli and colleagues, they demonstrated that antisense oligonucleotides  aos  are effective in restoring dystrophin expression in duchenne muscular dystrophy  dmd  patients through exon-skipping. however, the high doses required and the potential toxicity of aos limit their clinical applicability. to address these limitations, they developed a delivery system using nanoparticles  nps  composed of a poly methyl methacrylate   pmma  core and a random copolymer shell formed by units derived from n-isopropyl acrylamide  nipam  and reactive methacrylate-bearing cationic groups. this system improved the binding capacity of aos on the np surface through the introduction of various reactive functional groups. the study also revealed the potential of nps as nontoxic delivery vectors for aos, as lower doses of aos were required, reducing both the potential side effects and the duration of the administration schedule. a further study by bassi and colleag

In [ ]:
responseSrTrump_2 = adrian_steve_rags('sparse', 'who is Donald Trump?', False)

[2013  675  662  663  664]
in a study by capelli and colleagues, they demonstrated that antisense oligonucleotides  aos  are effective in restoring dystrophin expression in duchenne muscular dystrophy  dmd  patients through exon-skipping. however, the high doses required and the potential toxicity of aos limit their clinical applicability. to address these limitations, they developed a delivery system using nanoparticles  nps  composed of a poly methyl methacrylate   pmma  core and a random copolymer shell formed by units derived from n-isopropyl acrylamide  nipam  and reactive methacrylate-bearing cationic groups. this system improved the binding capacity of aos on the np surface through the introduction of various reactive functional groups. the study also revealed the potential of nps as nontoxic delivery vectors for aos, as lower doses of aos were required, reducing both the potential side effects and the duration of the administration schedule. a further study by bassi and colleag

In [ ]:
responseDrTrump_1 = adrian_steve_rags('dense', 'who is Donald Trump?', True)

[1614 1492  753 1231  330]
as of february 20, 2020, there have been over 72,000 confirmed cases of covid-19, with the majority in china. discrimination and prejudice, driven by fear or misinformation, have emerged in response to the outbreak. this has resulted in innocent people in china being unfairly blamed, and similar incidents have occurred in other countries. the formal names for the virus and disease have been established by the international committee on taxonomy of viruses and the world health organization, respectively. the outbreak has highlighted the negative side of human societies, as prejudice and bigotry have occurred with each huge epidemic in history. fear, prejudice, and discrimination can impede anti-covid-19 efforts, as they can discourage people from seeking medical help and prevent effective control measures from being implemented. open and transparent information, as well as administrative warnings, are necessary to reduce fear and discrimination.
Best match: ('

In [ ]:
responseDrTrump_2 = adrian_steve_rags('dense', 'who is Donald Trump?', False)

[1614 1492  753 1231  330]
as of february 20, 2020, there have been over 72,000 confirmed cases of covid-19, with the majority in china. discrimination and prejudice, driven by fear or misinformation, have emerged in response to the outbreak. this has resulted in innocent people in china being unfairly blamed, and similar incidents have occurred in other countries. the formal names for the virus and disease have been established by the international committee on taxonomy of viruses and the world health organization, respectively. the outbreak has highlighted the negative side of human societies, as prejudice and bigotry have occurred with each huge epidemic in history. fear, prejudice, and discrimination can impede anti-covid-19 efforts, as they can discourage people from seeking medical help and prevent effective control measures from being implemented. open and transparent information, as well as administrative warnings, are necessary to reduce fear and discrimination.
Best match: ('

## Evaluation

In [ ]:
user_queries = [
    'How do various SARS-CoV-2 variants affect the efficacy of current vaccines and treatments, and what strategies can be employed to enhance their effectiveness against emerging strains?',
    'What is the origin of SARS-CoV-2?',
    'Why do individuals experience varying severity of COVID-19 symptoms?',
    'How long does immunity last after infection or vaccination?',
    'Who is Donald Trump?'
    ]

retrievers = ['dense','sparse'] # fill with user queries

def producePred(retrievers, user_queries, augment_query):
  generated_responses = []
  produced_prompts = []
  for ret_type in retrievers:
    for query in user_queries:
      response = adrian_steve_rags(ret_type, query, augment_query)
      parts = response.split('<|assistant|>')
      generated_responses.append(parts[1])
      produced_prompts.append(parts[0])

  return produced_prompts, generated_responses

## Open ended/Borderline relevant Queries retrieved (10x What-If Experiments)

In [ ]:
produce_prompts_augmented, generated_responses_augmented = producePred(retrievers,user_queries, True)

[ 135  744 1744   52 1459]
the structural analysis of sars-cov receptor interactions has revealed that critical residue changes in sars-cov and host receptor proteins present major species barriers for viral infections and determine host range and cross-species transmission. sars-cov adapted to civet and human ace2 proteins by strengthening the binding affinity to their receptors through mutations in the receptor-binding domain  rbd  of its spike protein. mice support sars-cov infection at low levels because of the k353h residue change in their ace2, but rats are resistant to sars-cov infection because of the k353h and m82n double residue changes. through natural evolutions of its rbd at key positions, sars-cov overcame the species barriers between some animal species, but not others. a structure-based prediction of future sars-cov evolution suggests that if sars-cov were given the opportunity to keep infecting humans and evolving in human cells, it could evolve into a human-optimized 

In [ ]:
produce_prompts, generated_responses = producePred(retrievers,user_queries, False)

[ 135  744 1744   52 1459]
the structural analysis of sars-cov receptor interactions has revealed that critical residue changes in sars-cov and host receptor proteins present major species barriers for viral infections and determine host range and cross-species transmission. sars-cov adapted to civet and human ace2 proteins by strengthening the binding affinity to their receptors through mutations in the receptor-binding domain  rbd  of its spike protein. mice support sars-cov infection at low levels because of the k353h residue change in their ace2, but rats are resistant to sars-cov infection because of the k353h and m82n double residue changes. through natural evolutions of its rbd at key positions, sars-cov overcame the species barriers between some animal species, but not others. a structure-based prediction of future sars-cov evolution suggests that if sars-cov were given the opportunity to keep infecting humans and evolving in human cells, it could evolve into a human-optimized 

Zephyr (Huggingface) model is not perfect. Despite having 7B parameters model, it still was hallucinating and misunderstanding the prompt. ChatGPT 3.5/4o didn't cause any misunderstanding issues, and never hallucinated. If costs/funding was not an issue, ChatGPT 3.5/4o would have been integrated instead of the freely available Zephyr model.
* https://huggingface.co/HuggingFaceH4/zephyr-7b-beta
* https://huggingface.co/HuggingFaceH4/zephyr-7b-alpha/discussions/3
* https://haystack.deepset.ai/blog/guide-to-using-zephyr-with-haystack2

In [ ]:
!pip install rouge-score

## Build Rouge Scorer [ Temp Data ] - Ignore

In [ ]:
from rouge_score import rouge_scorer

# Sample reference and candidate summaries
reference_summary = """The provided contexts do not explicitly list the common symptoms of COVID-19. However, they discuss various aspects of COVID-19, SARS-CoV, and other viral infections. Below are 20 summarized findings relevant to understanding COVID-19 and related viral diseases:

Over 72,000 confirmed cases of COVID-19 were reported as of February 20, 2020, with most cases occurring in China [0].
Fear, prejudice, and misinformation have emerged alongside the outbreak, leading to discrimination [0].
COVID-19-related fear can discourage people from seeking medical help, which may impact control measures [0].
Open and transparent information dissemination is necessary to reduce fear and discrimination [0].
The SARS-CoV virus encodes a 7a protein that inhibits cellular protein synthesis [1].
The SARS-CoV 7a protein activates the p38 mitogen-activated protein kinase (MAPK) pathway [1].
The 7a protein may contribute to SARS-CoV cytopathogenesis by promoting apoptosis [1].
Additional proteins in SARS-CoV may also inhibit cellular translation, but their mechanisms are yet to be fully determined [1].
Idiopathic thrombocytopenic purpura (ITP) can be triggered by viral infections, including coronaviruses [2].
Most ITP cases present with minor mucosal bleeding, but severe cases can lead to intracranial hemorrhage [2].
ITP is thought to be caused by an autoimmune response that destroys platelets or inhibits their synthesis [2].
Mumps is a contagious viral disease transmitted via direct contact, droplets, and contaminated surfaces [3].
Mumps complications include orchitis, testicular atrophy, and, in rare cases, testicular tumors [3].
Mumps infection during early pregnancy has been linked to spontaneous abortion in some studies [3].
Mumps prevention relies on vaccination, primarily through the MMR vaccine [3].
Herpes simplex virus (HSV) hepatitis is a rare but serious cause of acute liver failure [4].
HSV hepatitis is difficult to diagnose antemortem but can be confirmed via liver biopsy [4].
Symptoms of HSV hepatitis include flu-like symptoms such as fever, myalgias, and abdominal pain [4].
Only 30-50% of patients with HSV hepatitis show characteristic herpetic skin lesions [4].
Early treatment with acyclovir improves outcomes in HSV hepatitis cases [4].
These findings provide indirect insights into viral infections, including COVID-19, but do not directly list common symptoms of COVID-19. For a comprehensive answer, direct reference to COVID-19 symptomatology from medical sources would be necessary."""

candidate_summary = """Based on the provided contexts, the common symptoms of COVID-19 include:

1. As of February 20, 2020, there have been over 72,000 confirmed cases of COVID-19, with the majority in China. (Context 0)
2. Symptoms of COVID-19 can include fever, myalgia, and fatigue. (Implied from the context of the first case report of COVID-19 in Wuhan, China, published in The Lancet on January 24, 2020) (Context 5)
3. COVID-19 can lead to pneumonia, with symptoms including coughing, shortness of breath, and breathing difficulties. (Context 5)
4. In severe cases, COVID-19 can result in acute respiratory distress syndrome (ARDS) and septic shock. (Context 5)
5. COVID-19 can cause loss of smell and taste, which has been reported as a symptom in some patients. (Context 5)
6. The virus that causes COVID-19 is called SARS-CoV-2, and the disease caused by the virus is called COVID-19. (Context 0)
7. The formal name for the disease caused by COVID-19 is COVID-19, which stands for Coronavirus Disease 2019. (Context 0)
8. Discrimination and prejudice have emerged in response to the COVID-19 outbreak, with innocent people in China being unfairly blamed. (Context 0)
9. Fear, prejudice, and discrimination can impede anti-COVID-19 efforts, as they can discourage people from seeking medical help and prevent effective control measures from being implemented. (Context 0)
10. Open and transparent information, as well as administrative warnings, are necessary to reduce fear and discrimination during the COVID-19 outbreak. (Context 0)
11. The COVID-19 virus belongs to the family of viruses known as coronaviruses, which includes other viruses that cause the common cold. (Context 5)
12. The COVID-19 virus can spread through respiratory droplets when an infected person coughs or sneezes, and can also spread through contact with contaminated surfaces. (Context 5)
13. COVID-19 can cause fatigue, muscle ache, and headache, similar to other viral infections. (Context 5)
14. COVID-19 can also cause diarrhea, which is less common but can be a symptom in some patients. (Context 5)
15. The COVID-19 virus can cause inflammation in the body, leading to symptoms such as fever and fatigue. (Context 5)
16. The COVID-19 virus can affect the respiratory system, leading to symptoms such as coughing, shortness of breath, and breathing difficulties. (Context 5)
17. COVID-19 can lead to pneumonia, which is more common in older adults and people with underlying medical conditions. (Context 5)
18. In severe cases, COVID-19 can result in acute respiratory distress syndrome (ARDS) and septic shock, which can be life-threatening. (Context 5)
19. COVID-19 can cause loss of smell and taste, which has been reported as a symptom in some patients. (Context 5)
20. The COVID-19 virus can spread to others through respiratory droplets when an infected person coughs or sneezes, and can also spread through contact with contaminated surfaces. (Context 5)

Based on these findings, it is clear that the symptoms of COVID-19 are varied and can include fever, myalgia, fatigue, coughing, shortness of breath, breathing difficulties, loss of smell and taste, diarrhea, inflammation, and in severe cases, acute respiratory distress syndrome (ARDS) and septic shock. The COVID-19 virus can also affect older adults and people with underlying medical conditions, and can spread through respiratory droplets and contact with contaminated surfaces. It is crucial to follow preventative measures, such as social distancing, wearing a mask, washing hands frequently, and practicing good hygiene, to prevent the spread of COVID-19."""

# Initialize the ROUGE scorer
scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)

# Compute ROUGE scores
scores = scorer.score(reference_summary, candidate_summary)

# Display the results
for metric, score in scores.items():
    print(f"{metric.upper()} - Precision: {score.precision:.4f}, Recall: {score.recall:.4f}, F1: {score.fmeasure:.4f}")

ROUGE1 - Precision: 0.3153, Recall: 0.4908, F1: 0.3839
ROUGE2 - Precision: 0.1188, Recall: 0.1852, F1: 0.1448
ROUGEL - Precision: 0.1610, Recall: 0.2507, F1: 0.1961


## Open ended queries: ChatGPT equivalent responses

### Sample prompt access

In [ ]:
produce_prompts[4]

'<|system|>\n  # This is a chat between a user and an AI assistant. The assistant follows strict rules when answering based on up to 5 provided contexts:\n\n  # 1. If the answer is found in the contexts, the assistant must provide a detailed response **with citations at the end of each point**.\n  # 2. The assistant must not add information beyond the provided contexts.\n  # 3. If no relevant answer is found, the assistant must respond with **\'Cannot extract findings from the provided contexts.\'** and **\'Summary is not available\'**.\n  # 4. **Citations must be formatted in IEEE style within square brackets []** at the end of each point.\n  # 5. Answers must be structured as a numbered list and **each point must contain its respective citation.**\n  # 6. Responses must be ranked according to the provided contexts.\n\n  # **Example format when findings can be extracted from the provided contexts:**\n  # Findings:\n  # **Example format when findings are available:**\n  # 1. Lorem ipsu

### Prompts were put on Chat GPT and the responses were recorded

https://chatgpt.com/share/67a370e3-996c-800b-b619-ed9e17a68828

In [ ]:
augmented_reference_summary_vaccines_dr =  """Findings:
1. Structural analysis of SARS-CoV receptor interactions reveals that residue changes in SARS-CoV and host receptor proteins create major species barriers, affecting viral infections, host range, and cross-species transmission [0].
2. SARS-CoV adapted to civet and human ACE2 proteins by increasing binding affinity through mutations in the receptor-binding domain (RBD) of its spike protein [0].
3. Mice support SARS-CoV infection at low levels due to the K353H residue change in their ACE2 receptor, while rats remain resistant because of both K353H and M82N residue changes [0].
4. Natural evolution of SARS-CoV’s RBD at key positions allowed the virus to overcome species barriers between some animals while still being restricted in others [0].
5. If SARS-CoV continued infecting and evolving in humans, structural predictions suggest it could optimize for human ACE2 with substantial binding affinity [0].
6. Convalescent plasma treatment for MERS-CoV relies on neutralizing antibodies, but donor availability with sufficiently high neutralizing antibody titers may be limited [1].
7. Monoclonal antibodies (mAbs) present an alternative to convalescent plasma and have shown efficacy in reducing viral titers and loads in lung tissues of rabbits and marmosets infected with MERS-CoV [1].
8. mAb treatment in MERS-CoV models led to reduced mortality and morbidity, surpassing the effects of hyperimmune plasma therapy [1].
9. The structural analysis of viral receptor interactions is crucial for predicting future virus evolution and potential vaccine or therapeutic targets [0].
10. The SARS-CoV 3b protein, which has an unknown function, is nuclear-localized and interferes with cell cycle regulation [2].
11. Overexpression of the 3b protein results in cell cycle arrest at the G0/G1 phase, leading to apoptosis in transfected cells [2].
12. The role of the 3b protein in inducing cell cycle arrest is consistent across different cell types, while apoptosis induction is cell-type specific [2].
13. Increased apoptosis and necrosis have been observed in SARS patients, affecting multiple organs, including the lung, spleen, liver, and lymph nodes [2].
14. Apoptotic cells in SARS patients include pneumocytes, lymphocytes, and monocytes, contributing to disease severity [2].
15. Two vaccine candidates, NDV-BC/S and NDV-VF/S, were tested for their immunogenicity and protective efficacy against SARS-CoV in primates [3].
16. These vaccine candidates showed restricted replication in non-human primates and minimal shedding in respiratory tract tissue samples [3].
17. A single dose of either vaccine candidate provided protective immunity against SARS-CoV challenge, with no observed clinical symptoms [3].
18. Both vaccine candidates induced a robust cell-mediated immune response, including SARS-CoV-specific CD4, CD8, and IFN-γ T cells [3].
19. There was no evidence of enhanced clinical disease symptoms in vaccinated animals, supporting their potential as a viable vaccine platform [3].
20. These vaccine candidates offer a potential vector for protection against SARS-CoV and other emerging viruses [3].  """
augmented_reference_summary_origin_SARS_dr =  """Findings:
1. MERS-CoV, a novel beta coronavirus, causes severe respiratory illness with a high case fatality rate and was first identified in 2012 in a patient in Saudi Arabia [1].
2. As of March 2015, there were 1,102 laboratory-confirmed cases of MERS-CoV, with a case fatality rate of 37.7% [0].
3. MERS-CoV belongs to the C lineage of the genus Betacoronavirus, with the bat species *Taphozous perforatus* identified as a probable natural reservoir [0].
4. The virus may spread indirectly to camels and humans through bat guano, saliva, or urine contaminating food and water resources in areas with palm orchards [0].
5. MERS-CoV primarily infects individuals in direct contact with camels, with other possible transmission routes including the consumption of unpasteurized camel milk [0].
6. MERS-CoV has been circulating for at least 31 years, with positive sera samples found from as early as 1983 in Somalia and Sudan [0].
7. MERS-CoV is prone to efficient replication in primate, pig, and goat cell cultures [0].
8. The functional receptor for MERS-CoV is dipeptidyl peptidase 4 (DPP4), also known as CD26 [0].
9. MERS-CoV transmission is less efficient than SARS-CoV, but it has a higher mortality rate, estimated between 35-50% [1].
10. Co-infection with other pathogens such as influenza A, parainfluenza, herpes simplex, and pneumococcus has been reported in MERS-CoV cases [0].
11. The prognosis for MERS-CoV is guarded, with respiratory failure and acute respiratory distress syndrome (ARDS) contributing to high mortality rates, especially in patients with comorbidities [1].
12. SARS-CoV encodes a protein, 3b, that interferes with cell cycle regulation and is localized in the nucleus [2].
13. Overexpression of the SARS-CoV 3b protein leads to cell cycle arrest in the G0/G1 phase and induces apoptosis in transfected cells [2].
14. The role of the 3b protein in inducing cell cycle arrest is conserved across different cell types, while its apoptotic effects are cell-type specific [2].
15. SARS-CoV infection results in increased apoptosis and necrosis, which are observed in multiple organs of affected patients [2].
16. Apoptotic cells in SARS patients have been identified in the lungs, spleen, liver, and lymph nodes, particularly affecting pneumocytes, lymphocytes, and monocytes [2].
17. SARS-CoV inhibits the activation of interferon beta (IFN-β), preventing early activation of the innate immune system [3].
18. The virus specifically blocks IFN-β gene transcription by interfering with interferon regulatory factor 3 (IRF-3) function, which may contribute to its high pathogenicity [3].
19. SARS-CoV avoids immune detection by blocking a molecular step between the nuclear transport of IRF-3 and its subsequent activation via hyperphosphorylation and dimerization [3].
20. SARS was first recognized in February 2003 in Hanoi, Vietnam, with a high attack rate among healthcare workers and an incubation period of 5-9 days [4].  """
augmented_reference_summary_COVID_symptoms_dr =  """Findings:
1. As of February 20, 2020, there were over 72,000 confirmed cases of COVID-19, with the majority in China [0].
2. Fear, prejudice, and discrimination have emerged in response to the outbreak, which can discourage people from seeking medical help and hinder control measures [0].
3. Open and transparent information, along with administrative warnings, are necessary to reduce fear and discrimination related to COVID-19 [0].
4. Human coronaviruses (CoVs) are a major cause of respiratory infections and outbreaks among elderly residents in long-term care facilities (LTCFs) [1].
5. Among 3,268 LTCF residents with positive respiratory virus specimens, OC43 was the most commonly identified human coronavirus, accounting for 52% of cases [1].
6. In 39% of cases among LTCF residents, human coronavirus infections were associated with respiratory outbreaks [1].
7. The majority (87%) of LTCF residents with human coronavirus infections had at least two common symptoms, including cough, runny nose/congestion, sore throat/hoarse voice, and fever [1].
8. Only 6% of LTCF residents with human coronavirus infections received antibiotic treatment for suspected secondary bacterial pneumonia [1].
9. The 30-day mortality rate among LTCF residents infected with human coronaviruses was 3.7% [1].
10. A 42-year-old female healthcare provider tested positive for MERS-CoV and presented with high-grade fever, productive cough, shortness of breath, and bony pains [2].
11. The patient had a history of left nephrectomy, cesarean section, abortion, and was newly diagnosed with type 2 diabetes mellitus [2].
12. Chest X-ray findings in the patient revealed bilateral infiltrates, more prominent on the right side, with significant improvement observed by day 13 [2].
13. The patient was treated with peginterferon alpha-2a, ribavirin, and methylprednisolone for MERS-CoV infection [2].
14. The patient developed new-onset diabetes insipidus and spontaneous intraventricular hemorrhage, raising questions about the incidence of sudden-onset intracranial hemorrhage in obese patients with diabetes insipidus [2].
15. The MERS-CoV PCR test remained positive even after one month of antiviral therapy, indicating the need for further studies on viral load, infection severity, and infection control [2].
16. Due to tonsillar herniation, no cerebrospinal fluid (CSF) sample for MERS-CoV testing could be obtained, raising the possibility of viral penetration into neuronal and glial cells [2].
17. Psychological distress, including post-traumatic stress disorder (PTSD), was observed in 6% of SARS survivors at both 1 and 3 months after hospital discharge [4].
18. SARS survivors who were healthcare workers, knew someone with SARS, or had a history of psychiatric consultation had a higher risk of psychological distress [4].
19. At 3 months post-discharge, PTSD symptom severity (intrusion, avoidance, hyperarousal) and anxiety and depression scores were significantly lower than at 1 month [4].
20. The lowest blood oxygen saturation during hospitalization was a significant predictor of PTSD symptoms, while perceived threat level was the most significant predictor of hyperarousal and anxiety scores [4].  """
augmented_reference_summary_immunity_dr="""Findings:
1. The viral shedding pattern of SARS-CoV differs from other respiratory viruses, with respiratory shedding peaking during the second week and lasting for more than six weeks in some cases [0].
2. Stool shedding of SARS-CoV occurs later than expected, with virus detection in stool specimens peaking at the beginning of the second week and remaining high over weeks 3-4 [0].
3. SARS-CoV can survive for at least 2-4 days at room temperature when mixed with diarrheal or normal stool specimens [0].
4. The poor isolation rate of SARS-CoV from stool may be due to the presence of toxic substances in stool or the interference of viral growth by immunoglobulin A (IgA) antibodies [0].
5. Positive virologic results in SARS-CoV patients are associated with more adverse outcomes, though this may be confounded by the fact that high-yield specimens, such as tracheal aspirates, are obtained from intubated patients [0].
6. A combination of stool samples and pooled throat and nasal swab specimens should be the preferred method for high-yield SARS-CoV detection [0].
7. The diagnostic yield of pooled throat and nasal swab specimens is higher compared to nasopharyngeal aspirates for SARS-CoV detection [0].
8. Throat washing samples are the least effective respiratory specimens for SARS-CoV detection [0].
9. The ability to diagnose SARS-CoV infection accurately needs improvement, with suggested methods including adding viremia levels to diagnostic algorithms and developing SARS-CoV-specific monoclonal antibodies [0].
10. The unusual shedding pattern of SARS-CoV should be considered when developing infection control strategies [0].
11. A single high dose of MDEF201 administered 8 weeks prior to infection provides a high degree of protection from death in a mouse model infected with vaccinia virus [1].
12. Lower doses of MDEF201 administered one day before infection are also effective against vaccinia virus in mice, though higher doses are required for equivalent protection when delivered to the nasal sinus compared to the lungs [1].
13. The extended prophylactic effect of MDEF201 against vaccinia virus in mice lasts at least 8 weeks after administration [1].
14. The adenovirus vector used to deliver MDEF201 remains present in treated mice 60 days after exposure, suggesting long-term effects even in the absence of measurable serum interferon levels [1].
15. High doses of vitamin C (3.0-4.0 g/day) during the common cold can aid faster recovery and reduce the duration of illness [2].
16. A combination of supplemental and therapeutic doses of vitamin C can relieve symptoms such as chest pain, fever, and chills, as well as shorten the duration of illness [2].
17. Large doses of vitamin C at the onset of a common cold can reduce its duration by approximately half a day and shorten indoor confinement by about 10 hours [2].
18. Vitamin C supplementation enhances immune system function, including antimicrobial activity, natural killer cell function, lymphocyte proliferation, and chemotaxis [2].
19. Vitamin C is the most effective water-soluble antioxidant in human plasma and is concentrated in leukocytes, where its levels decline rapidly during infections and stress [2].
20. The effectiveness of therapeutic doses of vitamin C for shortening the duration of the common cold varies by sex and dosage, with studies suggesting greater effectiveness in men [2]."""
augmented_reference_summary_trump_dr =  """Findings:
No answer found."""
augmented_reference_summary_vaccines_sr = """Findings:
1. Sequence variations in the spike (S) protein among different SARS-CoV strains pose potential problems for the development of effective vaccines due to antigenic protein sequence variation [0].
2. Rapid mutations in segments encoding the major antigenic proteins of SARS-CoV strains may lead to variations that restrict siRNA inhibitory effects, complicating vaccine development [0].
3. The target sequences of siRNAs (psr02 and psr03) that effectively inhibit SARS-CoV replication are conserved across all published SARS-CoV strains, suggesting a strategic focus on conserved regions may enhance vaccine efficacy [0].
4. Recombinant vector-based vaccines expressing the full-length S protein of late SARS-CoV strains are under development; however, these vaccines may induce antibodies that enhance infection by early human SARS-CoV and animal SARS-CoV-like viruses [1].
5. The receptor-binding domain (RBD) fragment of the S protein, consisting of multiple conformational neutralizing epitopes, is a promising vaccine candidate as it does not contain immunodominant sites that induce non-neutralizing antibodies, offering a safer alternative to full-length S protein vaccines [1].
6. Removal of the amino acid region 1153–1194 from the S protein can prevent the induction of infection-enhancing antibodies while still eliciting neutralizing antibodies, suggesting a strategy to avoid antibody-dependent enhancement (ADE) [1].
7. A previously unknown SARS virus genome sequence, SARS-CoV-CPSR-19, found in the ORF3b gene, significantly induces cell apoptosis, highlighting a potential target for understanding viral pathogenesis and improving therapeutic strategies [2].
8. SARS-CoV-CPSR-19’s role in infection or pathogenesis through the induction of cell apoptosis provides insights into the mechanisms of SARS-CoV infection, which could inform future therapeutic developments [2].
9. Differences in the receptor-binding domains (RBDs) of SARS-CoV and SARS-CoV-2, with SARS-CoV-2 RBD being cleaved by TMPRSS2 and cathepsin L in human cells, suggest different mechanisms of entry that may affect vaccine and treatment efficacy [3].
10. SARS-CoV-2’s ability to use human ACE2 as a receptor with higher efficiency compared to SARS-CoV indicates that vaccines targeting this interaction may be more critical for SARS-CoV-2 [3].
11. Recombinant RBD vaccines are considered safer and more efficacious against SARS-CoV variants with distinct genotypes due to the conserved nature of the RBD sequence [1].
12. Sequence conservation between different SARS-CoV strains in the target sequences of psr02 and psr03 siRNAs may help in maintaining the effectiveness of siRNA-based therapies despite genetic variability [0].
13. Mutations in the S glycoprotein, particularly in the neutralizing epitope region, have been linked to reduced vaccine efficacy, suggesting the need for continuous monitoring and updating of vaccine compositions [4].
14. Variations in the S protein, especially hydrophobic alterations, impact the effectiveness of immune responses, necessitating the design of vaccines that can accommodate such variations [4].
15. Strategies that involve truncation of specific amino acid regions in the S protein can mitigate the risk of antibody-dependent enhancement (ADE), thereby enhancing vaccine safety [1].
16. Understanding the role of SARS-CoV-CPSR-19 in inducing apoptosis can lead to the development of antiviral therapies targeting this mechanism to reduce virus-induced cell damage [2].
17. Vaccines focusing on the RBD of the S protein are less likely to induce non-neutralizing antibodies, reducing the risk of ADE and providing broader protection across SARS-CoV variants [1].
18. The finding that SARS-CoV-2 can be cleaved by both TMPRSS2 and cathepsin L but SARS-CoV cannot suggests potential therapeutic targets for SARS-CoV-2 that are not applicable to SARS-CoV [3].
19. Conservation of siRNA target sequences across SARS-CoV strains highlights the potential for siRNA therapies to remain effective despite genetic variations [0].
20. Differences in virus entry mechanisms between SARS-CoV and SARS-CoV-2, such as reliance on different proteases, underline the need for tailored therapeutic approaches targeting these pathways [3].  """
augmented_reference_summary_origin_SARS_sr = """Findings:
1. Electron microscopy and immunofluorescence staining demonstrated that both mature and immature dendritic cells (DCs) can be infected by SARS-CoV, indicating that cell types lacking ACE2 may also be susceptible, suggesting other receptors might be involved in virus entry [0].
2. SARS-CoV infection in DCs is incomplete, with viral genome expression but no increase in viral copies over six days, no virus budding, and no production of infectious virus in culture medium [0].
3. The infection of DCs by SARS-CoV did not induce significant cell death, as observed under light and electron microscopy or active caspase-3 assays, indicating that SARS-CoV’s immunosuppressive effect is not due to a direct cytopathic effect on DCs [0].
4. SARS-CoV infection did not upregulate the expression of MHC class I or MHC class II molecules on immature DCs, though DC maturation was still inducible by LPS, suggesting SARS-CoV does not impair DC maturation [0].
5. Sequence variation among different SARS-CoV strains poses challenges for the development of effective vaccines due to mutations in major antigenic proteins [1].
6. Two siRNAs (PSR02 and PSR03) effectively blocked SARS-CoV replication in Vero cells, targeting the RNA polymerase, which suggests their potential as alternative therapeutic agents [1].
7. The target sequences of PSR02 and PSR03 are conserved across different SARS-CoV strains, reducing the likelihood that sequence variation would limit their inhibitory effects [1].
8. The rapid mutation of the S protein among different SARS-CoV strains creates potential difficulties in developing a universal vaccine [1].
9. Structural analysis of SARS-CoV receptor interactions shows that residue changes in SARS-CoV and host receptor proteins create major species barriers, influencing host range and cross-species transmission [2].
10. SARS-CoV adapted to civet and human ACE2 proteins by strengthening the binding affinity through mutations in the receptor-binding domain (RBD) of the spike protein [2].
11. Mice support SARS-CoV infection at low levels due to the K353H residue change in their ACE2 receptor, whereas rats are resistant due to both K353H and M82N changes [2].
12. SARS-CoV overcame species barriers between some animal species through natural evolution of its RBD, but not all species [2].
13. Structural analysis suggests that if SARS-CoV were allowed to keep infecting humans, it could evolve into a human-optimized form with greater affinity for human ACE2 [2].
14. A novel sequence in the SARS-CoV genome, SARS-CoV-CPSR-19, located in the ORF3b gene, was identified and found to significantly induce cell apoptosis, a key feature of SARS-CoV pathogenesis [3].
15. RNA interference (RNAi) experiments confirmed that SARS-CoV-CPSR-19 plays a role in SARS-CoV infection and pathogenesis [3].
16. SARS-CoV-CPSR-19 can form a hairpin structure, and its melting temperature (~20°C) suggests that it may evade ligation during small RNA sequencing processes [3].
17. Another small viral RNA, SVRNA-N, was identified, though its exact role in SARS-CoV infection and pathogenesis remains unknown [3].
18. SARS-CoV and SARS-CoV-2 both have receptor-binding domains (RBDs) that bind to human ACE2, but SARS-CoV-2 also has an additional RBD that binds to bat ACE2 orthologs [4].
19. The spike glycoprotein of SARS-CoV-2, but not SARS-CoV, can be cleaved by TMPRSS2 in human airway epithelial cells, a prerequisite for efficient virus entry [4].
20. SARS-CoV-2 can be cleaved by cathepsin L (CTL) in human cells, a property not observed in SARS-CoV, facilitating its entry into human cells [4]."""
augmented_reference_summary_COVID_symptoms_sr =  """Findings:
1. The Liu-Hui-Wen model predicts that 15-30% of the world’s population will be infected with COVID-19, with the majority of infections occurring in Asia [1].
2. A study published in the *Journal of the American College of Cardiology* found that patients with COVID-19 who developed myocarditis had a higher risk of adverse outcomes, including heart failure and death, compared to those without myocarditis [2].
3. A study published in *The Lancet* found that patients with COVID-19 who developed acute respiratory distress syndrome (ARDS) had a higher risk of mortality and organ support compared to those without ARDS [2].
4. A study published in the *Journal of the American Medical Association* found that COVID-19 patients requiring intensive care unit (ICU) admission had a higher risk of mortality and prolonged hospital stays compared to those who did not require ICU admission [2].
5. A study published in the *Annals of Internal Medicine* found that patients with COVID-19 who developed acute kidney injury had a higher risk of mortality compared to those without acute kidney injury [2].
6. A study published in the *Journal of Hospital Medicine* found that COVID-19 patients who required mechanical ventilation had a higher risk of mortality and prolonged hospital stays compared to those who did not require mechanical ventilation [2].

Summary is not available beyond these six findings."""
augmented_reference_summary_immunity_sr =  """Findings:
1. A single-dose vaccination with vaccinia virus (VTT) through different routes (intranasal [i.n.], intraoral [i.o.], intramuscular [i.m.], and subcutaneous [s.c.]) induced long-lasting antivaccinia neutralizing antibody (NAb) responses in mice [0].
2. Higher levels (4.4-fold) of NAbs were induced via the mucosal i.n. and i.o. inoculations compared to the s.c. route, indicating that the route of vaccination influences the level of induced immunity [0].
3. Mucosal VTT vaccination (i.n. or i.o.) provided complete protection against viral challenges through mucosal routes, highlighting its effectiveness in mucosal immunity [0].
4. Mucosal VTT vaccination overcame preexisting antivaccinia immunity generated through prior skin or s.c., i.n., or i.o. routes of VTT vaccination, suggesting its potential for boosting immunity even in previously vaccinated individuals [0].
5. The route of VTT vaccination determines the level of long-lasting NAbs induced, indicating that different vaccination methods may have varying protective effects over time [0].
6. Influenza vaccination uptake among adults aged 50-64 in Hong Kong is strongly associated with being eligible for a free government vaccine and the willingness to receive it for free [1].
7. Perceiving severe or moderate symptoms when contracting the flu and the convenience of reaching a vaccination location are factors significantly influencing influenza vaccination uptake in this age group [1].
8. A lack of awareness exists among adults aged 50-64 in Hong Kong regarding their inclusion in the recommended priority group for influenza vaccination, with over 80% of cases and 93.9% of controls unaware of their eligibility [1].
9. The willingness of healthcare workers in Hong Kong to accept pre-pandemic influenza vaccination is low, with a response rate below 50% [2].
10. The acceptance of seasonal influenza vaccination among healthcare workers globally is low, with only 32.9% of healthcare workers in Hong Kong and 15.6% in the UK willing to receive pre-pandemic vaccines during the SARS outbreak [2].
11. The main barriers to influenza vaccination among healthcare workers include fear of side effects and doubts about vaccine efficacy [2].
12. A history of seasonal influenza vaccination and the perceived likelihood of being infected are factors associated with the intention to accept vaccination among healthcare workers [2].
13. Efforts to improve seasonal influenza vaccination uptake among healthcare workers should be part of pandemic preparedness plans, as disseminating correct information becomes more challenging during crises [2].
14. Public health agencies should provide more information to healthcare workers, particularly those with higher levels of anxiety and doubt, to improve vaccine acceptance [2].
15. Younger healthcare staff and those with less than five years of experience are more willing to accept vaccination, though prior experience with the SARS outbreak does not necessarily enhance willingness [2].
16. More than 90% of elderly individuals in institutions in Hong Kong received the influenza vaccine when delivered on-site, indicating the effectiveness of accessible vaccination programs [2].
17. There are common barriers to influenza vaccination across geographical regions and racial groups, suggesting a need for tailored vaccination campaigns [2].
18. Campaigns promoting vaccination should address the knowledge gap among staff and target groups for intervention to improve vaccine acceptance [2].
19. Further qualitative studies, such as focus groups or semi-structured interviews, should be conducted to supplement existing findings on vaccination acceptance [2].
20. A successful vaccination strategy should not only protect healthcare workers' health but also limit disease transmission between the healthcare sector and the community, as learned from the SARS outbreak [2].  """
augmented_reference_summary_trump_sr =  """No answer found."""


https://chatgpt.com/share/67a370e3-996c-800b-b619-ed9e17a68828

In [ ]:
reference_summary_vaccines_dr =  """Findings:

1. The provided contexts do not contain information on SARS-CoV-2 variants specifically.
2. No mention of how various SARS-CoV-2 variants affect the efficacy of current vaccines is found in the provided contexts.
3. The contexts discuss vaccines for SARS-CoV but not for SARS-CoV-2 variants.
4. Information on SARS-CoV vaccine candidates' immunogenicity in non-human primates and their ability to provide protective immunity is provided, but it is not related to SARS-CoV-2. [3]
5. The contexts do not discuss the impact of SARS-CoV-2 variants on current treatments.
6. No strategies for enhancing vaccine or treatment effectiveness against SARS-CoV-2 emerging strains are mentioned in the contexts.
7. Cannot extract findings from the provided contexts.

Summary is not available."""
reference_summary_origin_SARS_dr =  """Findings:

1. No information related to the origin of SARS-CoV-2 is available in the provided contexts.
2. The contexts only mention SARS-CoV and MERS-CoV, not SARS-CoV-2.
3. No answer found.

Summary is not available."""
reference_summary_COVID_symptoms_dr =  """Findings:

1. Fear, prejudice, and discrimination related to COVID-19 can impede effective control measures and discourage individuals from seeking medical help, which could influence the severity of symptoms due to delayed care [0].
2. In elderly residents of long-term care facilities, the majority of coronavirus cases show common symptoms like cough, runny nose, sore throat, and fever, but the overall treatment and outcomes are not significantly influenced by outbreaks or seasonality [1].
3. The severity of symptoms in COVID-19 patients may be related to their medical history, as a healthcare provider with pre-existing conditions like diabetes mellitus type 2 presented with worsening symptoms and developed complications such as diabetes insipidus and intraventricular hemorrhage [2].
4. MERS-CoV patients, including a case with a healthcare provider, tested positive for MERS-CoV for over a month, which may indicate that prolonged viral presence could impact the severity of symptoms and overall recovery [2].

Summary is not available."""
reference_summary_immunity_dr="""Findings:

1. In a mouse model, the extended prophylactic effect of mdef201 against vaccinia virus infections lasts at least 8 weeks after administration [1].
2. The adenovirus vector used to deliver mdef201 is still present in the snouts of mdef201-treated mice 60 days after exposure, suggesting a long-lasting protective effect [1].
3. The study of PRRSV vaccination in pigs shows that neutralizing antibodies are first detected at day 56 and remain at similar levels until challenge, with a subsequent boost after the challenge [4].

Summary is not available."""
reference_summary_trump_dr = """Findings:

1. Donald Trump's past tweets on public health emergencies have raised concerns about the potential for his reactions to define policy without consulting scientific and medical authorities [4].

Summary is not available."""
reference_summary_vaccines_sr =  """Findings:

1. Sequence conservation between different SARS-CoV strains is important in avoiding sequence variation that could restrict siRNA inhibitory effects, which may offer a strategy for developing vaccines or treatments effective across strains [0].
2. The target sequences of siRNAs psr02 and psr03, which can block SARS-CoV replication, are conserved across different SARS-CoV strains, suggesting that these sequences could be used to counteract mutations in emerging strains [0].
3. Genetic mutations in the segments encoding major antigenic proteins of SARS-CoV could pose challenges for vaccine development, as rapid mutations in the S protein lead to potential antigenic variation among strains, thus reducing vaccine effectiveness [0].
4. An ideal SARS-CoV vaccine should elicit potent neutralizing antibody responses against a broad spectrum of viral strains to remain effective against emerging variants, including strains with significant genetic divergence [1].
5. Recombinant RBD vaccines, using a fragment of the SARS-CoV S protein, have been proposed as a safer and more effective alternative to full-length S protein vaccines, as RBD does not induce non-neutralizing antibodies and is relatively conserved across variants [1].
6. The S protein of SARS-CoV with truncation at aa 1153 does not cause antibody-dependent enhancement of infection, suggesting that truncation could improve vaccine safety while maintaining efficacy against emerging strains [1].
7. A previously unknown sequence, SARS-CoV-CPSR-19, was found to significantly induce cell apoptosis and plays a role in SARS-CoV infection or pathogenesis, indicating potential targets for further exploration in vaccine or therapeutic development [2].
8. The identification of distinct receptor-binding domains (RBDs) in SARS-CoV and SARS-CoV-2 shows that viral binding to host cells may vary between strains, which can affect how well vaccines targeting these RBDs work across different variants [3].
9. The spike glycoprotein of SARS-CoV-2, but not that of SARS-CoV, can be cleaved by transmembrane serine protease 2 (TMPRSS2) and cathepsin L in human cells, which facilitates viral entry into human cells and could impact the effectiveness of treatments targeting these proteins [3].
10. A human ACE2-overexpressing cell line is permissive to SARS-CoV-2 infection, while SARS-CoV does not efficiently infect this cell line, highlighting differences in how SARS-CoV-2 variants may interact with host receptors, affecting vaccine and treatment strategies [3].
11. The differences in receptor interactions between SARS-CoV and SARS-CoV-2 variants could influence the success of vaccine and treatment strategies designed for one strain but not the other [3].
12. Mutations in the spike protein of SARS-CoV-2 may alter its ability to bind to human ACE2 receptors, influencing vaccine efficacy against new variants [3].
13. The emergence of SARS-CoV-2 variants with lower affinity for certain receptor orthologs, such as bat ACE2, suggests that zoonotic transmission and cross-species infection could complicate vaccine and treatment strategies [3].
14. The high genetic variability in the S protein of SARS-CoV strains may lead to differing immune responses, affecting the long-term efficacy of current vaccines and suggesting the need for continuous adaptation in vaccine development [1].
15. Recombinant vaccines expressing the full-length S protein of SARS-CoV have the potential to induce antibodies that may enhance infection by early human or animal SARS-CoV-like viruses, highlighting the importance of careful vaccine design to avoid immune enhancement [1].
16. Rapid mutations in SARS-CoV strains, particularly in regions encoding key proteins like the S protein, could require ongoing adjustments in vaccine formulations to maintain effectiveness against new variants [0].
17. Current SARS-CoV vaccine strategies must be adapted to account for sequence variation in the S protein across different strains, particularly given the potential for mutation-driven changes that could reduce vaccine efficacy [1].
18. The role of non-neutralizing antibodies in enhancing infection emphasizes the need for vaccines targeting conserved, neutralizing epitopes in order to minimize risks of immune enhancement from emerging strains [1].
19. Variations in the SARS-CoV strains' genetic makeup, especially in the spike protein, pose challenges for developing broadly effective treatments and vaccines, indicating a need for adaptive strategies [0].
20. Understanding the genetic variations across different SARS-CoV-2 strains and the functional consequences of these mutations will be crucial for enhancing the efficacy of vaccines and therapeutic treatments against new and emerging variants [0]."""
reference_summary_origin_SARS_sr = """Findings:

1. SARS-CoV-2 shares a receptor-binding domain (RBD) with SARS-CoV, but it also has a second RBD that binds to bat ACE2 orthologs, suggesting a connection to bat species. [4].
2. The spike glycoprotein of SARS-CoV-2 can be cleaved by transmembrane serine protease 2 (TMPRSS2) in human airway epithelial cells, which is crucial for efficient virus entry into human cells, a feature not observed in SARS-CoV. [4].
3. SARS-CoV-2 can also be cleaved by cathepsin L (CTL) in human cells, which is another key factor for efficient virus entry, further distinguishing it from SARS-CoV. [4].
4. A human ACE2-overexpressing cell line supports efficient replication of SARS-CoV-2 but not SARS-CoV, highlighting a difference in viral entry mechanisms between the two viruses. [4].
5. SARS-CoV-2 and SARS-CoV share an RBD that binds to human ACE2, but SARS-CoV-2 also has the ability to bind to certain bat ACE2 orthologs, indicating a potential cross-species transmission event involving bats. [4].
6. The spike protein of SARS-CoV-2 can bind to bat ACE2B, although with lower affinity compared to ACE2 orthologs from other bat species such as the horse shrew or Chinese horseshoe bat. [4].
7. SARS-CoV-2's spike protein is capable of binding to different bat ACE2 variants, reinforcing the idea that SARS-CoV-2 may have originated from bats and potentially adapted for human transmission. [4].
8. Structural analysis of SARS-CoV's receptor interactions showed that it adapted to bind to civet and human ACE2 proteins, suggesting a similar process for SARS-CoV-2 to adapt to human hosts. [2].
9. SARS-CoV's mutations in its receptor-binding domain allowed it to overcome species barriers, supporting the hypothesis that SARS-CoV-2 may have similarly evolved to infect humans. [2].
10. SARS-CoV-2 could evolve into a human-optimized form with a stronger affinity for human ACE2 if it continued to infect and evolve in human cells. [2].
11. SARS-CoV-2 has a receptor-binding domain that differs from that of SARS-CoV, allowing for a wider range of potential host species and cross-species transmission. [4].
12. SARS-CoV-2's ability to cleave its spike protein using TMPRSS2 and CTL in human cells suggests its distinct entry mechanism compared to SARS-CoV, indicating differences in their origins or evolutionary adaptations. [4].
13. SARS-CoV-2's origin may be linked to bat species, given the ability of the virus to interact with bat ACE2, although the exact origins and intermediate host remain unclear. [4].
14. SARS-CoV-2 shares certain key features with SARS-CoV, such as the use of ACE2 for cellular entry, but it also exhibits unique properties that could indicate a separate evolutionary lineage or adaptation. [4].
15. SARS-CoV-2 likely evolved from an ancestral virus that originally infected bats, based on the interaction with bat ACE2 and subsequent mutations enabling human transmission. [4].
16. SARS-CoV-2's mutation in its spike protein allows it to effectively bind to human ACE2, suggesting that the virus adapted to humans through natural evolution. [2].
17. The RBD of SARS-CoV-2 has undergone evolutionary changes that have allowed it to bind to human ACE2 with high affinity, indicating an adaptation process similar to SARS-CoV. [2].
18. SARS-CoV-2's ability to bind to different bat ACE2 variants highlights its potential zoonotic origin, likely from bat species, and its subsequent adaptation to humans. [4].
19. The natural evolution of SARS-CoV-2's receptor-binding domain and its ability to bind to human ACE2 suggests that it evolved specifically to infect humans after a zoonotic transmission event. [2].
20. The discovery of the SARS-CoV-2 genome and its similarities to SARS-CoV suggests that it may have originated from an earlier strain of SARS-like coronaviruses, with potential modifications for human-to-human transmission. [4]."""
reference_summary_COVID_symptoms_sr =  """Findings:

1. Patients with COVID-19 who developed myocarditis had a higher risk of adverse outcomes, including heart failure and death, compared to those without myocarditis. [2].
2. COVID-19 patients who developed acute respiratory distress syndrome (ARDS) had a higher risk of mortality and required more organ support compared to those without ARDS. [2].
3. COVID-19 patients who required intensive care unit (ICU) admission had a higher risk of mortality and prolonged hospital stays compared to those who did not require ICU admission. [2].
4. COVID-19 patients who developed acute kidney injury had a higher risk of mortality compared to those without acute kidney injury. [2].
5. Patients with COVID-19 who required mechanical ventilation had a higher risk of mortality and prolonged hospital stays compared to those who did not require mechanical ventilation. [2].
6. Fear, prejudice, and discrimination can impede COVID-19 control efforts, which may indirectly impact the health outcomes of individuals due to barriers in seeking medical help. [0].
7. The Liu-Hui-Wen model predicts that a significant portion of the global population will be infected with COVID-19, with higher infection rates occurring in Asia, which might affect the severity of symptoms in various regions. [1].
8. The increased demand for laboratory testing during the pandemic, including for COVID-19, suggests that timely detection and effective disease management can influence the severity of the disease in individuals. [1].
9. Pre-existing conditions such as heart disease, kidney injury, or respiratory issues can exacerbate the severity of COVID-19 symptoms. [2].
10. Patients who have higher levels of organ failure, such as heart, lung, or kidney failure, are at a higher risk of severe COVID-19 outcomes. [2].
11. The severity of COVID-19 symptoms can vary depending on the type of medical intervention needed, such as ICU admission or mechanical ventilation, indicating more severe cases are linked to higher treatment requirements. [2].
12. The availability of laboratory services and testing for COVID-19 has been crucial in detecting and managing the disease, which can influence the severity of symptoms through early intervention. [1].
13. The development of complications such as myocarditis or ARDS increases the severity and risk of death in COVID-19 patients. [2].
14. Patients who experience more severe forms of COVID-19 often require prolonged hospital stays and intensive care, which contributes to the worsening of their condition. [2].
15. The global COVID-19 pandemic has led to increased laboratory-based surveillance and response efforts, suggesting that monitoring and intervention are key factors in mitigating severity in infected individuals. [1].
16. Hospitalized COVID-19 patients who experience severe complications such as mechanical ventilation often have worse long-term outcomes, including higher mortality rates. [2].
17. Social discrimination and prejudice, though not directly related to symptom severity, can exacerbate health challenges, leading to delayed medical care, which can worsen the disease's progression in some individuals. [0].
18. As the pandemic has spread globally, varying access to healthcare resources and treatments across regions may affect the severity of COVID-19 symptoms experienced by individuals. [1].
19. Health disparities, including pre-existing chronic health conditions and access to care, may influence the severity of symptoms among different individuals, as seen in the increased risks faced by those requiring intensive interventions. [2].
20. The pandemic has emphasized the need for ongoing strategic discussions on preventing, detecting, and responding to COVID-19, which may help reduce the severity of outcomes through better preparedness and resource allocation. [1]."""
reference_summary_immunity_sr =  """Cannot extract findings from the provided contexts.
Summary is not available."""
reference_summary_trump_sr =  """Findings: No answer found."""


## Evaluation Methods: OPEN ENDED QUERIES

In [ ]:
#https://www.elastic.co/search-labs/blog/evaluating-rag-metrics
from nltk.translate.bleu_score import sentence_bleu, SmoothingFunction

augmented_references_dr = [augmented_reference_summary_vaccines_dr,augmented_reference_summary_origin_SARS_dr,augmented_reference_summary_COVID_symptoms_dr,augmented_reference_summary_immunity_dr,augmented_reference_summary_trump_dr]
augmented_references_sr = [augmented_reference_summary_vaccines_sr,augmented_reference_summary_origin_SARS_sr,augmented_reference_summary_COVID_symptoms_sr,augmented_reference_summary_immunity_sr,augmented_reference_summary_trump_sr]
references_dr = [reference_summary_vaccines_dr,reference_summary_origin_SARS_dr,reference_summary_COVID_symptoms_dr,reference_summary_immunity_dr,reference_summary_trump_dr]
references_sr = [reference_summary_vaccines_sr,reference_summary_origin_SARS_sr,reference_summary_COVID_symptoms_sr,reference_summary_immunity_sr,reference_summary_trump_sr]

scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
scores = scorer.score(reference_summary, candidate_summary)

def produceRougeBleu(references,retriever,generated_responses):

# Initialize DataFrames in dictionaries for sparse and dense retrievers
  metrics = ['rouge1', 'rouge2', 'rougeL']
  columns = ['precision', 'recall', 'fmeasure','bleu_scores']

  results = {
      'results': {metric: pd.DataFrame(columns=columns) for metric in metrics},
  }



  for id, reference in enumerate(references):
    scores = scorer.score(reference, generated_responses[id])

    reference_tokens = [reference.split()]  # BLEU expects list of lists
    generated_tokens = generated_responses[id].split()

    smoothing = SmoothingFunction().method1  # Smoothing to avoid zero scores
    bleu_score = sentence_bleu(reference_tokens, generated_tokens, smoothing_function=smoothing)

    # Process scores for each metric
    for metric, score in scores.items():
        metric_score = {
            "precision": score.precision,
            "recall": score.recall,
            "fmeasure": score.fmeasure,
            "bleu_scores":bleu_score
        }

    # Concatenate the new row to the corresponding DataFrame
        results['results'][metric] = pd.concat([
                results['results'][metric],
                pd.DataFrame([metric_score])
            ], ignore_index=True)
  outputResults(retriever,results)

In [ ]:
def outputResults(retriever,results):
  metrics = ['rouge1', 'rouge2', 'rougeL']
  columns = ['precision', 'recall', 'fmeasure','bleu_scores']

  print(retriever)
  for metric in metrics:
  # for column in columns:
    print('\n')
    print(metric)
    print('\n')
    print(results['results'][metric].mean())

In [ ]:
produceRougeBleu(augmented_references_dr,'dense',generated_responses_augmented[0:5])

<ipython-input-119-1d082f6f2dfd>:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results['results'][metric] = pd.concat([


dense


rouge1


precision      0.449681
recall         0.768088
fmeasure       0.497306
bleu_scores    0.171264
dtype: float64


rouge2


precision      0.254310
recall         0.448878
fmeasure       0.278877
bleu_scores    0.171264
dtype: float64


rougeL


precision      0.270956
recall         0.537935
fmeasure       0.298200
bleu_scores    0.171264
dtype: float64


In [ ]:
produceRougeBleu(augmented_references_sr,'sparse', generated_responses_augmented[5:10])

<ipython-input-119-1d082f6f2dfd>:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results['results'][metric] = pd.concat([


sparse


rouge1


precision      0.475959
recall         0.822688
fmeasure       0.513461
bleu_scores    0.179881
dtype: float64


rouge2


precision      0.291705
recall         0.620070
fmeasure       0.322720
bleu_scores    0.179881
dtype: float64


rougeL


precision      0.294216
recall         0.615449
fmeasure       0.323981
bleu_scores    0.179881
dtype: float64


In [ ]:
produceRougeBleu(references_dr,'dense',generated_responses[0:5])

dense


rouge1


precision      0.122106
recall         0.776183
fmeasure       0.203230
bleu_scores    0.026468
dtype: float64


rouge2


precision      0.052141
recall         0.397672
fmeasure       0.088385
bleu_scores    0.026468
dtype: float64


rougeL


precision      0.077481
recall         0.538747
fmeasure       0.130303
bleu_scores    0.026468
dtype: float64


<ipython-input-119-1d082f6f2dfd>:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results['results'][metric] = pd.concat([


In [ ]:
produceRougeBleu(references_sr,'sparse', generated_responses[5:10])

<ipython-input-119-1d082f6f2dfd>:43: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  results['results'][metric] = pd.concat([


sparse


rouge1


precision      0.313679
recall         0.496449
fmeasure       0.334944
bleu_scores    0.078938
dtype: float64


rouge2


precision      0.143812
recall         0.183877
fmeasure       0.153265
bleu_scores    0.078938
dtype: float64


rougeL


precision      0.158282
recall         0.298008
fmeasure       0.167921
bleu_scores    0.078938
dtype: float64


In [ ]:
#https://medium.com/@med.el.harchaoui/rag-evaluation-metrics-explained-a-complete-guide-dbd7a3b571a8#:~:text=Its%20considered%20as%20faithful%20if,range%20from%200%20to%201.
import numpy as np

def produceAnswerRelevance(references,generated,retriever):
  sim = []
  for i,reference in enumerate(references):
    reference_embedding = sentence_t5_model.encode(reference, convert_to_tensor=True)
    generated_embeddings = sentence_t5_model.encode(generated[i], convert_to_tensor=True)

    sim.append(util.cos_sim(reference_embedding, generated_embeddings).to('cpu'))

  sim_avg = np.array(sim).mean()
  print(f"{retriever}: {sim_avg}")

In [ ]:
produceAnswerRelevance(augmented_references_dr,generated_responses_augmented[0:5],'dense')

dense: 0.9022434949874878


In [ ]:
produceAnswerRelevance(augmented_references_sr,generated_responses_augmented[5:10],'sparse')

sparse: 0.9012987017631531


In [ ]:
produceAnswerRelevance(references_dr,generated_responses[0:5],'dense')

dense: 0.8576673269271851


In [ ]:
produceAnswerRelevance(references_sr,generated_responses[5:10],'sparse')

sparse: 0.8138960003852844
